# 6.1 - Reliability

**TODO:** Write useful documentation.

**Please note** that this notebook uses data from the repo at https://github.com/kkjaergaard/eos-rsa.

In [ ]:
import numpy as np
import sys
from matplotlib import pyplot as plt
from matplotlib import ticker as mpticker
import pandas as pd
import subprocess
import json
import time
import seaborn
seaborn.set()
import re
import os.path

sys.path.insert(0, "../src")
from rsa.series import RSA_Series_Logic
from rsa.dataset import RSA_Dataset

In [ ]:
filenames = [
    "delta x at isocenter.yaml",
    "delta y at isocenter.yaml",
    "delta z at isocenter.yaml",
    "delta x at entrance corner.yaml",
    "delta y at entrance corner.yaml",
    "delta z at entrance corner.yaml"
]

assessment_dd = np.arange(11) * 50

In [ ]:
filename_prefix = "../data/hip_phantom_full/yaml/"
df = pd.DataFrame()

for filename in filenames:
    for i, _from_dd in enumerate(assessment_dd):
        for j, to_dd in enumerate(assessment_dd):
            if i >= j:
                continue
                
            m = re.match(
                r'delta (x|y|z) at (isocenter|entrance corner)\.yaml',
                os.path.basename(filename)
            )
            axis = m.group(1)
            dimension = {"x": 0, "y": 1, "z": 2}[axis]
            position = m.group(2)
            rsa_axis = ["Ant-Post", "Lat-Med", "Inf-Sup"][dimension]                
                
            df = df.append(
                pd.Series({
                    "filename": filename,
                    "from": "d{}{:.0f}".format(axis, _from_dd),
                    "to": "d{}{:.0f}".format(axis, to_dd),
                    "true_displacement": (to_dd - _from_dd),
                    "axis": axis,
                    "dimension": dimension,
                    "rsa_axis": rsa_axis,
                    "position": position
                }),
                ignore_index=True
            )

df["dimension"] = pd.to_numeric(df["dimension"], downcast='integer')
print("Constructed {} rows".format(df.shape[0]))
df.head()

In [ ]:
t0 = time.time()

progress_every_n = 20

def run_assessment(series, total: int):
    q, r = divmod(int(series.name), progress_every_n)
    if r == 0 and q > 0:
        print("Processed {} ({:.1f}%) in {:.0f}s".format(
            q * progress_every_n,
            q * progress_every_n / total * 100,
            time.time() - t0
        ))
    
    cmd = [
        "python3",
        "../bin/find-motion.py",
        "--from",
        series["from"],
        "--to",
        series["to"],
        "--reference-segment",
        "acetabular beads",
        filename_prefix + series["filename"],
        "femoral beads"
    ]
    
    result = json.loads(subprocess.check_output(cmd))[0]
    return pd.Series({
        "measured_displacement": result["t"][series["dimension"]] * 1000,
        "t": result["t"],
        "R": result["R"],
        "ME": result["ME"],
    }, name=series.name)

df_assessments = df.apply(run_assessment, axis=1, args=(df.shape[0],))
df_assessments = df_assessments.join(df)
df_assessments["displacement_error"] = df_assessments["measured_displacement"] - df_assessments["true_displacement"]

df_assessments.head()

In [ ]:
#df_assessments.to_csv("../data/hip_phantom_full/results/reliability.csv")
#df_assessments = pd.read_csv("../data/hip_phantom_full/results/reliability.csv")

In [ ]:
def func(group):
    return pd.Series({
        "axis": group["axis"].iloc[0],
        "rsa_axis": group["rsa_axis"].iloc[0],
        "position": group["position"].iloc[0],
        "mean": group["displacement_error"].mean(),
        "std": group["displacement_error"].std(),
        "errors": group["displacement_error"].tolist()
    })

df_results = df_assessments.groupby("filename").apply(func).reset_index().sort_values(
    by=["position", "axis"],
    ascending=[False, True]
)

df_results

In [ ]:
x = []
cell_text = []
row_labels = [
    "movement",
    "phantom position", 
    "average error",
    "SD",
    "95% CI"
]

## calculate standard deviation for all errors
#all_errors_list = [v.df["dd"].tolist() for v in validations]
#all_errors = []
#for l in all_errors_list:
#    all_errors += l
#sigma_all = np.std(all_errors)
#n_all = len(all_errors)
#print(n_all, sigma_all)

for index, row in df_results.iterrows():
    x.append(row["errors"])
    cell_text.append([
        row["rsa_axis"],
        row["position"],
        "{:.1f} μm".format(row["mean"]),
#        "{:.1f} μm".format(
#            stats.t.ppf(1 - 0.05/2, dd.shape[0]-1) * math.sqrt(
#                sigma_all*sigma_all/n_all + dd.std()*dd.std()/dd.shape[0]
#            ) * 1000
#        ),
        "{:.1f} μm".format(
            row["std"]
        ),
        "{:.1f} μm".format(
            row["std"] * 1.96
        ),
#        "{:.1f} μm".format(
#            dd.std() / math.sqrt(dd.shape[0]) * 1000 * stats.t.ppf(1 - 0.05/2, dd.shape[0]-1)
#        ),
    ])

# transpose cell_text (thanks to https://stackoverflow.com/a/6473724)
cell_text = list(map(list, zip(*cell_text)))
    
plt.style.use("seaborn-whitegrid")
plt.figure(figsize=(10, 6))
plt.title("Error between true and measured movement", fontweight="bold")
    
lw = 2
plt.boxplot(
    x,
    boxprops=dict(linewidth=lw),
    whiskerprops=dict(linewidth=lw),
    medianprops=dict(linewidth=lw),
    capprops=dict(linewidth=lw)
)
plt.xticks([])
plt.ylabel("error (μm)")

the_table = plt.table(
    cellText=cell_text,
    rowLabels=row_labels,
    loc='bottom',
    edges="open"
)
plt.subplots_adjust(left=0.2, bottom=0.25)
plt.savefig("../data/esults/hip_phantom_full/results/Error between true and measured movement.png", dpi=300)
plt.show()